In [2]:
pip install --upgrade opencv-python


In [10]:
import tensorflow as tf
import numpy as np
from PIL import Image

# Load pre-trained MobileNetV2 model
model = tf.keras.applications.MobileNetV2(weights='imagenet', include_top=True)

# Load and preprocess images, converting to RGB format
image_paths = ["Rabbit.jpeg", "dog.jpeg","human.jpeg"]
images = [np.array(Image.open(path).resize((224, 224)).convert('RGB')) for path in image_paths]
images = [tf.keras.applications.mobilenet_v2.preprocess_input(img) for img in images]

# Perform inference on each image
predictions = [model.predict(np.expand_dims(img, axis=0)) for img in images]
predicted_labels = [tf.keras.applications.mobilenet_v2.decode_predictions(pred)[0] for pred in predictions]

# Display results
for i, labels in enumerate(predicted_labels):
    print(f"Image {i+1}:")
    for label in labels:
        print(f"{label[1]}: {label[2]*100:.2f}%")
    print()


1/1 [==============================] - 0s 56ms/step
Image 1:
hare: 64.11%
Angora: 15.17%
wood_rabbit: 1.63%
fox_squirrel: 0.36%
hamster: 0.34%

Image 2:
dingo: 31.47%
Eskimo_dog: 15.04%
Great_Dane: 3.27%
German_shepherd: 2.90%
Saluki: 2.65%

Image 3:
military_uniform: 11.46%
Indian_cobra: 2.63%
sunglass: 2.42%
drumstick: 2.09%
cleaver: 2.09%



In [18]:
from IPython.display import display, Javascript, Image
from google.colab.output import eval_js
from base64 import b64decode
import cv2
import numpy as np
import tensorflow as tf

# Load pre-trained MobileNetV2 model
model = tf.keras.applications.MobileNetV2(weights='imagenet', include_top=True)
class_names = tf.keras.utils.get_file('ImageNetLabels.txt','https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt')
class_names = np.array(open(class_names).read().splitlines())

def take_photo_and_classify(quality=0.8):
    js = Javascript('''
        async function takePhoto(quality) {
          const div = document.createElement('div');
          const capture = document.createElement('button');
          capture.textContent = 'Capture';
          div.appendChild(capture);

          const video = document.createElement('video');
          video.style.display = 'block';
          const stream = await navigator.mediaDevices.getUserMedia({video: true});

          document.body.appendChild(div);
          div.appendChild(video);
          video.srcObject = stream;
          await video.play();

          // Resize the output to fit the video element.
          google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

          // Wait for Capture to be clicked.
          await new Promise((resolve) => capture.onclick = resolve);

          const canvas = document.createElement('canvas');
          canvas.width = video.videoWidth;
          canvas.height = video.videoHeight;
          canvas.getContext('2d').drawImage(video, 0, 0);
          stream.getVideoTracks()[0].stop();
          div.remove();
          return canvas.toDataURL('image/jpeg', quality);
        }
    ''')

    display(js)
    data = eval_js('takePhoto({})'.format(quality))
    binary = b64decode(data.split(',')[1])

    # Save the captured image
    filename = 'photo.jpg'
    with open(filename, 'wb') as f:
        f.write(binary)

    # Load and preprocess the captured image
    img = tf.keras.preprocessing.image.load_img(filename, target_size=(224, 224))
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0)
    img_array = tf.keras.applications.mobilenet_v2.preprocess_input(img_array)

    # Perform inference
    predictions = model.predict(img_array)
    predicted_label_index = np.argmax(predictions)
    predicted_label = class_names[predicted_label_index]

    # Display classification result
    print("Predicted label:", predicted_label)

try:
    take_photo_and_classify()
except Exception as err:
    print(str(err))


<IPython.core.display.Javascript object>

1/1 [==============================] - 1s 933ms/step
Predicted label: impala
